![Image of Yaktocat](https://oepfelbaum.com/wp_live/wp-content/themes/appletree/dist/img/logo-main_ed18093c87353e6715bcc40c699c37f9.png)

# Oe LuKB Market Risk - Jupyter Notebook

**Author**: Fabian Arter <br>
**Date**: 2022-01-29 - 2022-05-01

The aim of this notebook is the explain the main concepts of market risk.

## Volatility & Return Distribution


<div class="alert alert-block alert-info">
<b>Input:</b> Please choose here for which titles you would like to see historical data. 
</div>



   Check **Yahoo Finance** for all tickers: [https://finance.yahoo.com](https://finance.yahoo.com/lookup) 

Enter in the below `data.frame` the desired tickers (incl. a friendly name, which can be chosen by you)

In [43]:
input.tickers.df = data.frame(ticker=c("ROG.SW", 
                                       "NESN.SW", 
                                       "NOVN.SW", 
                                       "^SSMI"),
                               friendly.name=c("Roche", 
                                               "Nestle", 
                                               "Novartis", 
                                               "SMI")
                              )


<div class="alert alert-block alert-info">
<b>Input:</b> Please choose here the start and end date of the historical price data. 
</div>



In [44]:
start.date = as.Date("2010-01-01")
last.date  = Sys.Date() # today 

In [45]:
print(input.tickers.df)
message(paste("start date:",start.date))
message(paste("last date: ",last.date))

   ticker friendly.name
1  ROG.SW         Roche
2 NESN.SW        Nestle
3 NOVN.SW      Novartis
4   ^SSMI           SMI


start date: 2010-01-01
last date:  2022-05-08


### Variance vs Volatility
The variance indicates the risk of an asset or of an entire portfolio, it is the standard deviation $\sigma$ (volatility) of returns squared $\sigma^2$. 


### Volatility of a portfolio
The risk of a portfolio can be quantified by computing the variance of the portfolio return, which is equal to $w^T\Sigma w$ where $w$ is the already introduced weight vector and the $\Sigma$ is the covariance matrix.


### Volatility: Codes Examples

In [ ]:
install.packages("ggplot2")
install.packages("BatchGetSymbols")
install.packages("dplyr")
install.packages("reshape2")
library(ggplot2)
library(BatchGetSymbols)
library(dplyr)

#' etlFinData
#'
#' @param start.date Start Date of the historical price data
#' @param end.date  End Date of the historical price data
#' @param input.tickers.df Data Frame with the products we wish to have the prices, this includes the ticker symbol and a friendly name
#' @return a list with two data frames: cumulated.returns.data.long and cumulated.returns.data.long
#' @export
etlFinData <- function(start.date=as.Date("2018-01-01"), 
                       last.date =Sys.Date(),
                       input.tickers.df = data.frame(ticker=c("LUKN.SW","AIR.PA"),
                                                     friendly.name=c("LUKN.SW","AIRBUS"))
) {
  
  # load data via BatchGetSymbols
  daily.returns.data.long <- BatchGetSymbols::BatchGetSymbols(
    tickers      = as.character(input.tickers.df$ticker),
    first.date   = start.date,
    last.date    = last.date,
    freq.data    = "daily",
    type.return  = "log",
    cache.folder = file.path(tempdir(), 'BGS_Cache') )$df.tickers
  
  
  # add friendly names
  daily.returns.data.long <-  merge(daily.returns.data.long, input.tickers.df, by="ticker")
  
  daily.returns.data.long[is.na(daily.returns.data.long$ret.adjusted.prices),]$ret.adjusted.prices <- 0
  daily.returns.data.long <- na.omit(daily.returns.data.long)
  
  daily.returns.data.long <- data.frame(ref.date        = daily.returns.data.long$ref.date,
                                        friendly.name   = as.character(daily.returns.data.long$friendly.name),
                                        price.adjusted  = daily.returns.data.long$price.adjusted,
                                        daily.return    = daily.returns.data.long$ret.adjusted.prices)
  
  daily.returns.data.wide <- reshape2::dcast(daily.returns.data.long,  ref.date ~ friendly.name, value.var = "daily.return")
  daily.returns.data.wide <- na.omit(daily.returns.data.wide)
  
  # cumulated returns wide
  cumulated.returns.data.wide  <- if(nrow(input.tickers.df) !=1) {
    as.data.frame(cbind(ref.date = daily.returns.data.wide$ref.date, apply(daily.returns.data.wide[,2:ncol(daily.returns.data.wide)], 2, cumsum)))
    
  }
    

  # cumulated returns long
  cumulated.returns.data.long <- if(nrow(input.tickers.df) !=1) {
    cumulated.returns.data.long               <- reshape2::melt(cumulated.returns.data.wide, id.vars=1, measure.vars = 2:ncol(cumulated.returns.data.wide))
    cumulated.returns.data.long$ref.date      <- as.Date(cumulated.returns.data.long$ref.date, origin = "1970-01-01")
    cumulated.returns.data.long$name          <- as.character(cumulated.returns.data.long$variable) ; cumulated.returns.data.long$variable <- NULL
    cumulated.returns.data.long$cumul.return  <- cumulated.returns.data.long$value ; cumulated.returns.data.long$value <- NULL
    cumulated.returns.data.long
  } else {
    data.frame(ref.date = daily.returns.data.long$ref.date, name = daily.returns.data.long$friendly.name, cumul.return=cumsum(daily.returns.data.long$daily.return))
  }
  
  cumulated.returns.data.wide$ref.date      <- as.Date(as.numeric(cumulated.returns.data.wide$ref.date), origin = "1970-01-01")
  
  return(list(daily.returns.data.long     = daily.returns.data.long,
              daily.returns.data.wide     = daily.returns.data.wide,
              cumulated.returns.data.long = cumulated.returns.data.long,
              cumulated.returns.data.wide = cumulated.returns.data.wide))
  
}    

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done


In [ ]:
PF.daily.return <- etlFinData(start.date = start.date,
                             
                              input.tickers.df = input.tickers.df)

PF.daily.return.wide <- PF.daily.return$daily.returns.data.wide
PF.daily.return.long <- PF.daily.return$daily.returns.data.long
PF.daily.cumul.long  <- PF.daily.return$cumulated.returns.data.long 



In [ ]:


# Shapiro-Wilk Test for Normality, 
# Null hypothesis: The data is normally distributed. If p> 0.05, normality can be assumed
random.returns.example <- rnorm(5000)
hist(random.returns.example)
shapiro.test(random.returns.example)

# HISTOGRAM LUKB
  ggplot(PF.daily.return.wide, aes(x=LUKN.SW)) +
  geom_histogram(colour="white", fill="#7997FF", alpha=0.6, position = 'identity', bins=25, aes(y=..density..)) +
  scale_fill_manual(values="white") + theme(legend.title = element_blank()) +
  stat_function(fun = dnorm, args = list(mean = mean(PF.daily.return.wide$LUKN.SW), sd = sd(PF.daily.return.wide$LUKN.SW))) +
  ggtitle("Histogram of Daily Returns")  + ylab("count")

 # QQ Plot - LUKB
 ggplot(PF.daily.return.wide) +
  stat_qq(aes(sample = LUKN.SW),colour = "#7997FF") + stat_qq_line(aes(sample = LUKN.SW)) +
  ggtitle("QQ Plot - LUKN.SW") + xlab("theoretical") + ylab("sample")



   # HISTOGRAM Bitcoin
   ggplot(PF.daily.return.wide, aes(x=`Bitcoin (USD)`)) +
    geom_histogram(colour="white", fill="#F763E0", alpha=0.6, position = 'identity', bins=25, aes(y=..density..)) +
    scale_fill_manual(values="white") + theme(legend.title = element_blank()) +
    stat_function(fun = dnorm, args = list(mean = mean(PF.daily.return.wide$`Bitcoin (USD)`), sd = sd(PF.daily.return.wide$`Bitcoin (USD)`))) +
    ggtitle("Histogram of Daily Returns")  + ylab("count")
  
  # QQ Plot - BITCOIN
  ggplot(PF.daily.return.wide) +
  stat_qq(aes(sample = `Bitcoin (USD)`),colour = "#F763E0") + stat_qq_line(aes(sample = `Bitcoin (USD)`)) +
  ggtitle("QQ Plot - Bitcoin (USD)") + xlab("theoretical") + ylab("sample")


## Volatility

In [ ]:

# Cumulated return plot
ggplot(PF.daily.cumul.long, aes(y=cumul.return, x=ref.date, color=name)) + geom_line(size=0.3)  

# Vola plot
ggplot(PF.daily.return.long, aes(y=daily.return, x=ref.date)) + geom_line(size=0.3)  + facet_grid(~ friendly.name)
  
PF.daily.return.long %>% group_by(friendly.name) %>% 
  summarise(min = min(daily.return)
            ,max = max(daily.return)
            ,mean = mean(daily.return)
            ,sd = sd(daily.return)
            ,n = n()
            ,q05 = quantile(daily.return, .05)
            ,q01 = quantile(daily.return, .01)
            ,q95 = quantile(daily.return, .95)
            ,q99 = quantile(daily.return, .99)
            ,shapiro.normality.pvalue = as.numeric(shapiro.test(daily.return)[2])
            ) 



In [ ]:
library(zoo)

SMI <- PF.daily.return.wide %>%select(ref.date, SMI)
Nestle <- PF.daily.return.wide %>%select(ref.date, Nestle)
Roche <- PF.daily.return.wide %>%select(ref.date, Roche)
Novartis <- PF.daily.return.wide %>%select(ref.date, Novartis)

names(SMI)[names(SMI) == "SMI"] <- "Return"
names(Nestle)[names(Nestle) == "Nestle"] <- "Return"
names(Roche)[names(Roche) == "Roche"] <- "Return"
names(Novartis)[names(Novartis) == "Novartis"] <- "Return"

SMI$Rolling60 <- c(c(rep(NA, 59)), rollapply(SMI$Return, 60, sd)) * sqrt(250)
SMI$Rolling120 <- c(c(rep(NA, 119)), rollapply(SMI$Return, 120, sd)) * sqrt(250)
SMI$Rolling250 <- c(c(rep(NA, 249)), rollapply(SMI$Return, 250, sd)) * sqrt(250)
SMI$Rolling500 <- c(c(rep(NA, 499)), rollapply(SMI$Return, 500, sd)) * sqrt(250)
SMI$Rolling750 <- c(c(rep(NA, 749)), rollapply(SMI$Return, 750, sd)) * sqrt(250)
SMI$Return <- NULL
SMI <- reshape2::melt(SMI, id.vars=1, measure.vars = 2:5)
ggplot(SMI, aes(y=value, x=ref.date, color=variable)) + geom_line(size=0.3) + ggtitle("SMI")

Nestle$Rolling60 <- c(c(rep(NA, 59)), rollapply(Nestle$Return, 60, sd)) * sqrt(250)
Nestle$Rolling120 <- c(c(rep(NA, 119)), rollapply(Nestle$Return, 120, sd)) * sqrt(250)
Nestle$Rolling250 <- c(c(rep(NA, 249)), rollapply(Nestle$Return, 250, sd)) * sqrt(250)
Nestle$Rolling500 <- c(c(rep(NA, 499)), rollapply(Nestle$Return, 500, sd)) * sqrt(250)
Nestle$Rolling750 <- c(c(rep(NA, 749)), rollapply(Nestle$Return, 750, sd)) * sqrt(250)
Nestle$Return <- NULL
Nestle <- reshape2::melt(Nestle, id.vars=1, measure.vars = 2:5)
ggplot(Nestle, aes(y=value, x=ref.date, color=variable)) + geom_line(size=0.3) + ggtitle("Nestle")

Roche$Rolling60 <- c(c(rep(NA, 59)), rollapply(Roche$Return, 60, sd)) * sqrt(250)
Roche$Rolling120 <- c(c(rep(NA, 119)), rollapply(Roche$Return, 120, sd)) * sqrt(250)
Roche$Rolling250 <- c(c(rep(NA, 249)), rollapply(Roche$Return, 250, sd)) * sqrt(250)
Roche$Rolling500 <- c(c(rep(NA, 499)), rollapply(Roche$Return, 500, sd)) * sqrt(250)
Roche$Rolling750 <- c(c(rep(NA, 749)), rollapply(Roche$Return, 750, sd)) * sqrt(250)
Roche$Return <- NULL
Roche <- reshape2::melt(Roche, id.vars=1, measure.vars = 2:5)
ggplot(Roche, aes(y=value, x=ref.date, color=variable)) + geom_line(size=0.3) + ggtitle("Roche")

Novartis$Rolling60 <- c(c(rep(NA, 59)), rollapply(Novartis$Return, 60, sd)) * sqrt(250)
Novartis$Rolling120 <- c(c(rep(NA, 119)), rollapply(Novartis$Return, 120, sd)) * sqrt(250)
Novartis$Rolling250 <- c(c(rep(NA, 249)), rollapply(Novartis$Return, 250, sd)) * sqrt(250)
Novartis$Rolling500 <- c(c(rep(NA, 499)), rollapply(Novartis$Return, 500, sd)) * sqrt(250)
Novartis$Rolling750 <- c(c(rep(NA, 749)), rollapply(Novartis$Return, 750, sd)) * sqrt(250)
Novartis$Return <- NULL
Novartis <- reshape2::melt(Novartis, id.vars=1, measure.vars = 2:5)
ggplot(Novartis, aes(y=value, x=ref.date, color=variable)) + geom_line(size=0.3) + ggtitle("Novartis")

## Correlation

In [ ]:
library(zoo)

SMI <- PF.daily.return.wide %>%select(ref.date, SMI)
Nestle <- PF.daily.return.wide %>%select(ref.date, Nestle)
Nestle$ref.date <-NULL
SMI_Nestle = cbind(SMI, Nestle)
SMI_Nestle$RollingCorr60 <- c( c(rep(NA, 59)), rollapply(SMI_Nestle[-1], 60, function(x) cor(x[,1],x[,2]), by.column=FALSE))
SMI_Nestle$RollingCorr120 <- c( c(rep(NA, 119)), rollapply(SMI_Nestle[-1], 120, function(x) cor(x[,1],x[,2]), by.column=FALSE))
SMI_Nestle$RollingCorr250 <- c( c(rep(NA, 249)), rollapply(SMI_Nestle[-1], 250, function(x) cor(x[,1],x[,2]), by.column=FALSE))
SMI_Nestle$RollingCorr500 <- c( c(rep(NA, 499)), rollapply(SMI_Nestle[-1], 500, function(x) cor(x[,1],x[,2]), by.column=FALSE))
SMI_Nestle$RollingCorr750 <- c( c(rep(NA, 749)), rollapply(SMI_Nestle[-1], 750, function(x) cor(x[,1],x[,2]), by.column=FALSE))
SMI_Nestle$Nestle <- NULL
SMI_Nestle$SMI <- NULL                                
SMI_Nestle <- reshape2::melt(SMI_Nestle, id.vars=1, measure.vars = 2:5)                                                         
ggplot(SMI_Nestle, aes(y=value, x=ref.date, color=variable)) + geom_line(size=0.3) + ggtitle("SMI-Nestle")
                                                           
SMI <- PF.daily.return.wide %>%select(ref.date, SMI)
Novartis <- PF.daily.return.wide %>%select(ref.date, Novartis)
Novartis$ref.date <-NULL
SMI_Novartis = cbind(SMI, Novartis)
SMI_Novartis$RollingCorr60 <- c( c(rep(NA, 59)), rollapply(SMI_Novartis[-1], 60, function(x) cor(x[,1],x[,2]), by.column=FALSE))
SMI_Novartis$RollingCorr120 <- c( c(rep(NA, 119)), rollapply(SMI_Novartis[-1], 120, function(x) cor(x[,1],x[,2]), by.column=FALSE))
SMI_Novartis$RollingCorr250 <- c( c(rep(NA, 249)), rollapply(SMI_Novartis[-1], 250, function(x) cor(x[,1],x[,2]), by.column=FALSE))
SMI_Novartis$RollingCorr500 <- c( c(rep(NA, 499)), rollapply(SMI_Novartis[-1], 500, function(x) cor(x[,1],x[,2]), by.column=FALSE))
SMI_Novartis$RollingCorr750 <- c( c(rep(NA, 749)), rollapply(SMI_Novartis[-1], 750, function(x) cor(x[,1],x[,2]), by.column=FALSE))
SMI_Novartis$Novartis <- NULL
SMI_Novartis$SMI <- NULL                                
SMI_Novartis <- reshape2::melt(SMI_Novartis, id.vars=1, measure.vars = 2:5)                                                         
ggplot(SMI_Novartis, aes(y=value, x=ref.date, color=variable)) + geom_line(size=0.3) + ggtitle("SMI-Novartis") 
                                                             
SMI <- PF.daily.return.wide %>%select(ref.date, SMI)
Roche <- PF.daily.return.wide %>%select(ref.date, Roche)
Roche$ref.date <-NULL
SMI_Roche = cbind(SMI, Roche)
SMI_Roche$RollingCorr60 <- c( c(rep(NA, 59)), rollapply(SMI_Roche[-1], 60, function(x) cor(x[,1],x[,2]), by.column=FALSE))
SMI_Roche$RollingCorr120 <- c( c(rep(NA, 119)), rollapply(SMI_Roche[-1], 120, function(x) cor(x[,1],x[,2]), by.column=FALSE))
SMI_Roche$RollingCorr250 <- c( c(rep(NA, 249)), rollapply(SMI_Roche[-1], 250, function(x) cor(x[,1],x[,2]), by.column=FALSE))
SMI_Roche$RollingCorr500 <- c( c(rep(NA, 499)), rollapply(SMI_Roche[-1], 500, function(x) cor(x[,1],x[,2]), by.column=FALSE))
SMI_Roche$RollingCorr750 <- c( c(rep(NA, 749)), rollapply(SMI_Roche[-1], 750, function(x) cor(x[,1],x[,2]), by.column=FALSE))
SMI_Roche$Roche <- NULL
SMI_Roche$SMI <- NULL                                
SMI_Roche <- reshape2::melt(SMI_Roche, id.vars=1, measure.vars = 2:5)                                                         
ggplot(SMI_Roche, aes(y=value, x=ref.date, color=variable)) + geom_line(size=0.3) + ggtitle("SMI-Roche")
                                                          
                                                          Novartis <- PF.daily.return.wide %>%select(ref.date, Novartis)
Roche <- PF.daily.return.wide %>%select(ref.date, Roche)
Roche$ref.date <-NULL
Novartis_Roche = cbind(Novartis, Roche)
Novartis_Roche$RollingCorr60 <- c( c(rep(NA, 59)), rollapply(Novartis_Roche[-1], 60, function(x) cor(x[,1],x[,2]), by.column=FALSE))
Novartis_Roche$RollingCorr120 <- c( c(rep(NA, 119)), rollapply(Novartis_Roche[-1], 120, function(x) cor(x[,1],x[,2]), by.column=FALSE))
Novartis_Roche$RollingCorr250 <- c( c(rep(NA, 249)), rollapply(Novartis_Roche[-1], 250, function(x) cor(x[,1],x[,2]), by.column=FALSE))
Novartis_Roche$RollingCorr500 <- c( c(rep(NA, 499)), rollapply(Novartis_Roche[-1], 500, function(x) cor(x[,1],x[,2]), by.column=FALSE))
Novartis_Roche$RollingCorr750 <- c( c(rep(NA, 749)), rollapply(Novartis_Roche[-1], 750, function(x) cor(x[,1],x[,2]), by.column=FALSE))
Novartis_Roche$Roche <- NULL
Novartis_Roche$Novartis <- NULL                                
Novartis_Roche <- reshape2::melt(Novartis_Roche, id.vars=1, measure.vars = 2:5)                                                         
ggplot(Novartis_Roche, aes(y=value, x=ref.date, color=variable)) + geom_line(size=0.3) + ggtitle("Novartis-Roche")
                                                               
Novartis <- PF.daily.return.wide %>%select(ref.date, Novartis)
Nestle <- PF.daily.return.wide %>%select(ref.date, Nestle)
Nestle$ref.date <-NULL
Novartis_Nestle = cbind(Novartis, Nestle)
Novartis_Nestle$RollingCorr60 <- c( c(rep(NA, 59)), rollapply(Novartis_Nestle[-1], 60, function(x) cor(x[,1],x[,2]), by.column=FALSE))
Novartis_Nestle$RollingCorr120 <- c( c(rep(NA, 119)), rollapply(Novartis_Nestle[-1], 120, function(x) cor(x[,1],x[,2]), by.column=FALSE))
Novartis_Nestle$RollingCorr250 <- c( c(rep(NA, 249)), rollapply(Novartis_Nestle[-1], 250, function(x) cor(x[,1],x[,2]), by.column=FALSE))
Novartis_Nestle$RollingCorr500 <- c( c(rep(NA, 499)), rollapply(Novartis_Nestle[-1], 500, function(x) cor(x[,1],x[,2]), by.column=FALSE))
Novartis_Nestle$RollingCorr750 <- c( c(rep(NA, 749)), rollapply(Novartis_Nestle[-1], 750, function(x) cor(x[,1],x[,2]), by.column=FALSE))
Novartis_Nestle$Nestle <- NULL
Novartis_Nestle$Novartis <- NULL                                
Novartis_Nestle <- reshape2::melt(Novartis_Nestle, id.vars=1, measure.vars = 2:5)                                                         
ggplot(Novartis_Nestle, aes(y=value, x=ref.date, color=variable)) + geom_line(size=0.3) + ggtitle("Novartis-Nestle") 
                                                                
Roche <- PF.daily.return.wide %>%select(ref.date, Roche)
Nestle <- PF.daily.return.wide %>%select(ref.date, Nestle)
Nestle$ref.date <-NULL
Roche_Nestle = cbind(Roche, Nestle)
Roche_Nestle$RollingCorr60 <- c( c(rep(NA, 59)), rollapply(Roche_Nestle[-1], 60, function(x) cor(x[,1],x[,2]), by.column=FALSE))
Roche_Nestle$RollingCorr120 <- c( c(rep(NA, 119)), rollapply(Roche_Nestle[-1], 120, function(x) cor(x[,1],x[,2]), by.column=FALSE))
Roche_Nestle$RollingCorr250 <- c( c(rep(NA, 249)), rollapply(Roche_Nestle[-1], 250, function(x) cor(x[,1],x[,2]), by.column=FALSE))
Roche_Nestle$RollingCorr500 <- c( c(rep(NA, 499)), rollapply(Roche_Nestle[-1], 500, function(x) cor(x[,1],x[,2]), by.column=FALSE))
Roche_Nestle$RollingCorr750 <- c( c(rep(NA, 749)), rollapply(Roche_Nestle[-1], 750, function(x) cor(x[,1],x[,2]), by.column=FALSE))
Roche_Nestle$Nestle <- NULL
Roche_Nestle$Roche <- NULL                                
Roche_Nestle <- reshape2::melt(Roche_Nestle, id.vars=1, measure.vars = 2:5)                                                         
ggplot(Roche_Nestle, aes(y=value, x=ref.date, color=variable)) + geom_line(size=0.3) + ggtitle("Roche-Nestle")                                                                
                                                         


## Covariance

### Sample Covariance
The sample covariance is defined as follows
\begin{equation}
s_{xy}=\frac{1}{n-1}\sum_{i=1}^{n}(x_{i}-\bar{x})(y_{i}-\bar{y}))
\end{equation}

### Correlation


### Covariance of Portfolio (Covariance Matrix)
The portfolio's covariance is defined as follows

\begin{multline}
\sigma^{2}_p=Var(R_p) 
= Var\left ( \sum_{i=1}^{k}w_i*R_i(t) \right ) 
=Cov\left ( \sum_{i=1}^{k}w_i*R_i(t),\sum_{j=1}^{k}w_j*R_j(t) \right ) \\ 
=\sum_{i=1}^{k}\sum_{j=1}^{k}w_i w_jCov(R_i(t),R_j(t)) 
=w^T\Sigma w
\end{multline}

where $\Sigma$ is the $k \times k$ covariance matrix with entries \(\sigma\) = Cov\( Ri,Rj \).


#### Covariance Matrix Decomposition
A covariance matrix can be decomposed into a volatility vector \(\sigma\) and a correlation matrix $R$:

We first create a diagonal standard deviation matrix $S$, which looks like this:



$S=\begin{pmatrix}
\sigma _{1} &0  &... &0 \\ 
 0& \sigma _{2} &... &0 \\ 
 \vdots &  &  \ddots &0 \\ 
0& ...\ & ...\ & \sigma _{n}
\end{pmatrix}$

 
and the correlation matrix $R$:


$R=\begin{pmatrix}
1 & \rho _{12} &...  &\rho _{1n}\\ 
 \rho _{21} &1& ... &\rho _{2n} \\ 
 \vdots &  &  \ddots & \\ 
\rho _{n1}& ...\ & ...\ & 1
\end{pmatrix}$


So our covariance matrix $\Sigma$ is generated by first multiplying the correlation matrix with the diagonal standard deviation matrix and then again multiplying by the transposed diagonal volatility matrix (as however $S$ is a diagonal matrix, it does not matter whether we take the transposed matrix or not, so $S=S^{^{T}}$). 

$\Sigma =SRS$

This decomposition can be helpful if we want to control the specific inputs, i.e. the correlation or the volatilites. 


### Covariance: Code Examples

In [ ]:
install.packages("ggcorrplot")
library(ggcorrplot)

PF.daily.return.data <- etlFinData(start.date=as.Date("2021-01-01"),
                                input.tickers.df = data.frame(ticker=c("^IXIC","AAPL","FB","INTC","MSFT","ABNB","ADBE","AMZN","EA"),
                                                              friendly.name=c("Nasdaq","Apple","Facebook","Intel","Microsoft","AirBnB","Adobe","Amazon","EA"))) 
  
  PF.daily.return.data.wide <- PF.daily.return.data$daily.returns.data.wide
  
  PF.correl.matrix <- cor(PF.daily.return.data.wide[,2:ncol(PF.daily.return.data.wide)])
  library(ggcorrplot)
  
  ggcorrplot(
    PF.correl.matrix,
    hc.order = TRUE,
    #type = "lower",
    outline.color = "white",
    ggtheme = ggplot2::theme_gray,
    colors = c("#6D9EC1", "white", "#E46726"),
    lab = TRUE
  )

# Note that the slope of a linear regression is not the same as the correlation coefficient (!)
  ggplot(PF.daily.return.data.wide, aes(x=Nasdaq, y=Microsoft)) + geom_point() +
    geom_smooth(method=lm, se=FALSE)
 
# Correlation example Microsoft, Nasdaq
  correl.manual = cov(PF.daily.return.data.wide$Microsoft, PF.daily.return.data.wide$Nasdaq) / (sd(PF.daily.return.data.wide$Microsoft)*sd(PF.daily.return.data.wide$Nasdaq))
  print(correl.manual)

## Value at Risk ##
As defined by Wilmott (2007) <em>Value at Risk (VaR)</em> 
"is an estimate, with a given degree of confidence, of how much on can lose from one's portfolio over a given time horizon"

### Parametrical (delta-normal) VaR ###

#### Parametrical Value at Risk for single Assets ####
The parametrical Value at Risk (VaR) is also called analytical VaR, it is based on a mean-variance or delta-normal approach. 
It basically only requires an estimation of the mean and the volatility of the returns, which are assumed to be normally distributed.

$\Delta$ represents the quantity of the asset hold with price $S$ and volatility $\sigma$.

\begin{equation}
VaR^{param}=-\sigma \Delta S(\delta t)^{1/2}\alpha (1-c)
\end{equation}


where $\alpha\left ( \cdot  \right )$ is the inverse cumulative distribution function for the standardized Normal distribution. 
$(\delta t)$ is the time horizon (usually one to five days) and $c$ is the degree of confidence (e.g. 0.95 or 0.99). 
The mean of the distribution is assumed to be zero (which is reasonable for short time periods, such as in the VaR, which are usually ranging from one to five days).

If we assume that the returns are not autocorrelated, we can simply scale the volatility by $\sqrt{t}$, if we e.g. have a daily volatility $\sigma$ of 1.5\% and a confidence level of 99\% and a holding period of 5 days we get:

$VaR_{5d @0.99}^{param}=-0.015 \times \Delta S \times \sqrt{5} \times 2.33$

#### Parametrical Value at Risk for Portfolios #### 
The parametric VaR of a portfolio can be calculated in the following way:

\begin{equation}
VaR_{PF}^{param}=\upsilon \Pi  \upsilon ^{T}
\end{equation}

where $\upsilon$ is the vector of the single Assets VaR figures and $\Pi $ is the correlation matrix of the assets' returns.

### Historical (Non-parametrical) Value at Risk ###
The nonparametric or historical VaR is simply derived from historical data. This has the advantage that we do not estimate any return distribution functions or worry about volatilities.  

\begin{equation}
VaR_{1-\alpha}^{hist}=R_\alpha
\end{equation}

$VaR_{1-\alpha}$ is the VaR at a confidence level of $1-\alpha$ or in other words the $\alpha$ th's worst return is our VaR. If we have 100 daily returns and we want to know the 95\% VaR, we simply look at the 5th worst daily return, which corresponds to the historical VaR at a 95\% confidence level.

### Conditional Value at Risk or Expected Shortfall ###
Conditional Value-at-Risk or also commonly known as Expected Shortfall, is the amount $\alpha$ which is the conditional expectation of losses above a specified probability level $\beta$. The $\beta$-VaR of a portfolio is the lowest amount $\alpha$ such that, the loss will not be above $\alpha$.  The $\beta$-CVaR
is the conditional expectation of losses exceeding $\alpha$. So the CVaR is the mean of all returns that are beyond (or "worse" than) VaR. Therefore CVaR is always more severe than VaR, which shows also the focus on the more extreme tail risks that are not captured by VaR (as with VaR we do not know how much worse it can go once we have breached VaR). 

Mathematically CVaR can be defined as

\begin{equation}
CVaR_{\alpha }(X)=\frac{1}{\alpha }\int_{0}^{\alpha }VaR_{\beta }(X)d\beta 
\end{equation}

or alternatively 

\begin{equation}
CVaR_{\alpha }(X)=E\left [ X|X\geq VaR_{\alpha }(X) \right ]
\end{equation}

As with VaR, CVaR can be parametrical or non-parametrical. 

### Value at Risk: Code Examples

#### Value at Risk: Functions

In [ ]:

# VaR Functions

#' @param daily.volatility The applied volatility for the shift
#' @param VaR.confidence Confidence level of the Value at Risk (VaR) shift
#' @param VaR.period.days Holding period in days for the VaR shift
#' @return a shifted price according to volatility, VaR confidence level and VaR period
#' @name VaRparamSingle
#' @examples
#' VaR(price=100, position=1, daily.volatility=0.01, VaR.confidence=0.99, VaR.period.days=5)
#' @export
VaRparamSingle <- function(daily.volatility, VaR.confidence=0.99, VaR.period.days=5) {
  param.var.single = -daily.volatility*qnorm(VaR.confidence)*sqrt(VaR.period.days)
  return(param.var.single)
}


#' @title VaRparamPortfolio
#' @description This function returns the Value at Risk of a Portfolio
#' @param vector with individual.var
#' @param correlation.matrix
#' @return a Portfolio VaR
#' @name VaRparamPortfolio
#' @export
VaRparamPortfolio <- function(individual.var.vector, correlation.matrix) {

  portfolio.var <- sqrt(individual.var.vector  %*% correlation.matrix  %*% individual.var.vector)
  return(portfolio.var)
}


#' @param VaR.confidence Confidence level of the Value at Risk (VaR) shift
#' @param historical.return.ts - Historical returns timeseries of single stock
#' @param price Original price (pre-shift) per unit
#' @param position size of the position

VaRHistSimSingle <- function(historical.return.ts, VaR.confidence) {

hs.var.single <- as.numeric(quantile(historical.return.ts,1-VaR.confidence))                       
return(hs.var.single)
}    

#' @param daily.returns.ts - Historical returns timeseries of single stock  
#' @param VaR.confidence Confidence level of the Value at Risk (VaR) shift
#' @param lambda - time decay factor
  VaRHistSimWeightSingle <- function(daily.returns.ts, lambda, VaR.confidence) {
    
    K = length(daily.returns.ts)
    hist.return.data <- data.frame(return=daily.returns.ts, days.ago.n=c(1:K))
    hist.return.data$time.weight <- (1-lambda)*lambda^(hist.return.data$days.ago.n-1)/(1-lambda^K)
    hist.return.data2 <- hist.return.data[order(hist.return.data$return),]
    hist.return.data2$weight.cumul <- cumsum(hist.return.data2$time.weight)
    var.hist.time.weighted <- approx(x=hist.return.data2$weight.cumul, y=hist.return.data2$return, xout=1-VaR.confidence)$y
    return(var.hist.time.weighted)
  }   
   


EWMAvolaEstim <- function(daily.returns.ts,lambda=0.94,time.period=100) {
    
    ewma.vector <- (1-lambda)*lambda^(c(1:time.period)-1)/(1-lambda^time.period)
    estimation.vola.yesterday <- sqrt(sum(daily.returns.ts[2:(time.period+1)]^2*ewma.vector))
    estimation.return2        <- daily.returns.ts[2]^2
    
    estimation.vola.today <- sqrt(lambda*estimation.vola.yesterday^2 + (1-lambda)*estimation.return2)
    return(estimation.vola.today)
  }



# Conditional VaR / Expected Shortfall Using Historical Returns
# description calculates the historical Conditional VaR / Expected Shortfall of a portfolio or an asset
# param daily.returns Daily historical returns of a portfolio or single asset
# param alpha.cvar Alpha of the CVaR, this is the confidence level from which on the average of the tail risk is being calculated
# return cvar CVaR of the specific portfolio or asset with the set alpha
histCVaRcalc <- function(asset.weights=1, daily.returns.data.wide, alpha.cvar) {
  i.num = ceiling(alpha.cvar * nrow(daily.returns.data.wide))
  portfolio.returns <- as.matrix(daily.returns.data.wide[,2:length(daily.returns.data.wide)]) %*% c(asset.weights)
  portfolio.returns <- sort(portfolio.returns)
  cvar = mean(portfolio.returns[1:i.num])
  return(cvar)
}


#### Code Examples - Execution

In [ ]:
# VaR (1 title) - LuKB

# 1d VaR, 99% confidence

historical.return.ts <- PF.daily.return.wide$LUKN.SW
VaR.confidence  = 0.99 # confidence level
VaR.period.days = 1    # holding period
time.period     = 100  # num historical days 
lambda          = 0.94 # time decay factor


# Example - Parametric VaR 
param.var.single <- VaRparamSingle(
                                daily.volatility=sd(historical.return.ts[1:time.period]),
                                VaR.confidence=VaR.confidence,
                                VaR.period.days=VaR.period.days)

# Example - Hist Single VaR 
hs.var.single <- VaRHistSimSingle(historical.return.ts[1:time.period], VaR.confidence)

# Example - Hist Weight Single VaR 
hs.weight.var.single <- VaRHistSimWeightSingle(historical.return.ts[1:time.period], lambda, VaR.confidence)


# Example - EWMA
ewma.vola <- EWMAvolaEstim(historical.return.ts,lambda,time.period)
ewma.var <- -ewma.vola*qnorm(VaR.confidence)


result.table <- data.frame(var.model=c("parametric","historical simulation", "HS time weighted", "EWMA"),
                           var.result=c(param.var.single,hs.var.single,hs.weight.var.single, ewma.var))


print(result.table)



In [ ]:

# Auto Correlation Function plot 
plot(acf(historical.return.ts^2))



### EWMA plot
lambda <- 0.95
K <- 100
ewma <- (1-lambda)*lambda^(c(1:K)-1)/(1-lambda^K)
plot(ewma)



#### Code Examples - VaR Backtesting

In [ ]:
# Value at Risk Backtesting
  
  return.lukb <- historical.return.ts
  length.data <- length(return.lukb)
  daily.vola.lukb <- vector(length=length(PF.daily.return.wide$LUKN.SW)-29)
   for (i in 30:length.data) {
    daily.vola.lukb[i] <- (sd(return.lukb[i-29:i]))
   }
  
  backtesting.data <- data.frame(daily.return=return.lukb[30:length.data],
                         daily.vola.rolling=daily.vola.lukb[30:length.data])
  
  backtesting.data$var95.up <- backtesting.data$daily.vola.rolling*qnorm(0.95)
  backtesting.data$var99.up <- backtesting.data$daily.vola.rolling*qnorm(0.99)
  backtesting.data$var95.down <- backtesting.data$daily.vola.rolling*-qnorm(0.95)
  backtesting.data$var99.down <- backtesting.data$daily.vola.rolling*-qnorm(0.99)
  backtesting.data$date <- PF.daily.return.wide$ref.date[30:nrow(PF.daily.return.wide)]
  
  ggplot(data=backtesting.data, aes(x=date, y=daily.return)) + geom_point(size=1,color="black") + geom_line(size=0.3, color="#7997FF", linetype=1) +
  geom_ribbon(aes(ymin=var95.down,ymax=var95.up), linetype=2, alpha=0.3, colour="red") +
  geom_ribbon(aes(ymin=var99.down,ymax=var99.up), linetype=2, alpha=0.2, colour="darkred") +
  ggtitle("VaR Backtesting - LUKN.SW (95% vs 99% Confidence)")



#### Code Examples - Principal Component Analysis (PCA)

In [ ]:

return.dat.multiple <- PF.daily.return.data.wide[,2:ncol(PF.daily.return.data.wide)]
head(return.dat.multiple)
plot(princomp(return.dat.multiple))
  
  pca.result <- princomp(return.dat.multiple)
  summary(pca.result)


## Option Valuation & Greeks
### Black Scholes

In [ ]:
# Black Scholes

BlackScholesValuation <- function(S, K, T, vola, rf) {
  
  d1 = (log(S/K) + (rf+0.5*vola^2) * T) / (vola*sqrt(T))
  d2 = d1 - vola*sqrt(T)
  
  call.value = S*pnorm(d1) - K*exp(-rf*T)*pnorm(d2)
  put.value  = K*exp(-rf*T)*pnorm(-d2) - S*pnorm(-d1) 
    
  call.delta = pnorm(d1)
  put.delta  = call.delta-1
    
  results <- list(call.value=call.value, put.value=put.value, call.delta= call.delta, put.delta = put.delta )
  return(results)
}


### Implied Vola helper
volaOptimizer <- function(vola, option.price, S, K, rf, T, option.type) {
  if(option.type=="call") {abs(option.price - BlackScholesValuation(S, K, T, vola, rf)$call.value)} else {
    abs(option.price - BlackScholesValuation(S, K, T, vola, rf)$put.value) }
}

## find Implied Vola
findImpliedVola <- function(option.price, S, K, rf, T, option.type) {
implied.vola <- optimize(volaOptimizer, interval = c(0, 2), option.price = option.price, S = S, K = K, rf = rf, T = T, option.type = "call")$minimum
return(implied.vola)
}

# Monte Carlo One Step Simulation 
MCOneStepSimulation <- function(S, dt, vola, rf, num.simulations, method) {
  
  if(!(method %in% c("Euler","Ito"))) {stop("method must be Euler or Ito")} 
  
  set.seed(20180425)
  
  if(method=="Euler") {
    ST = S + S*rf*dt + vola*S*sqrt(dt)*rnorm(num.simulations)
  }
  
  if(method=="Ito") {
    ST = S*exp((rf-0.5*vola^2)*dt+sqrt(dt)*vola*rnorm(num.simulations)) 
  }    
  return(ST)
}

EulerPathSimulator <- function(num.steps=252, num.simulations, num.discrete.steps=12, S, vola, rf) {
  
  
  sim.list <- vector("list", num.simulations)  
  
  for(i in 1:num.simulations) {
    single.simulation.df    <- data.frame(id.simulation=i,day=c(1:num.steps), S.cont=NA, S.discrete=NA)
    single.simulation.df$S.cont                <- (1+cumsum(c(0,log(1+rf)/num.steps + vola/sqrt(num.steps)*rnorm(num.steps-1))))*S
    single.simulation.df$S.discrete            <- ifelse(single.simulation.df$day %% num.discrete.steps ==0, single.simulation.df$S.cont, NA)
    single.simulation.df$S.discrete[1]         <- S
    single.simulation.df$S.discrete            <- na.approx(single.simulation.df$S.discrete, method="constant")
    single.simulation.df$S.cont.mean.arith     <- cumsum(single.simulation.df$S.cont)/single.simulation.df$day
    single.simulation.df$S.discrete.mean.arith <- cumsum(single.simulation.df$S.discrete)/single.simulation.df$day
    single.simulation.df$S.cont.mean.geom      <- exp(cumsum(log(single.simulation.df$S.cont)) / single.simulation.df$day)
    single.simulation.df$S.discrete.mean.geom  <- exp(cumsum(log(single.simulation.df$S.discrete)) / single.simulation.df$day)
    
    sim.list[[i]] <- single.simulation.df
    }
  
  sim.df <- plyr::rbind.fill(sim.list)
  
  
return(data.frame(sim.df))
}


# Monte Carlo European Option Valuation
MCEuropeanOptionValuation <- function(S, K ,dt, T, vola, rf, num.simulations, method) {
  
  simulated.ST <-  MCOneStepSimulation(S=S, dt=dt, vola=vola, rf=rf, num.simulations = num.simulations, method = method)
  call.value   <- exp(-rf)*mean(pmax(simulated.ST-K,0))
  put.value    <- exp(-rf)*mean(pmax(K-(simulated.ST),0))
  result.df    <- data.frame(call.value=call.value, put.value=put.value,num.simulations=num.simulations)
  return(result.df)
}




In [ ]:
#install.packages("zoo")
library(zoo)
library(ggplot2)


####################
# SET PARAMETERS
S    = 100 # stock price
K    = 100 # strike price
T    = 1 # time to expiry
vola = 0.2 # sigma
rf   = 0.05 # risk free rate
dt   = 1
method = "Euler"

# BLACK SCHOLES VALUATION
BS.option.value <- BlackScholesValuation(S=S, K=K, T=T, vola=vola, rf=rf)
print(BS.option.value)


# Delta 
S.shift = seq(0.5*S,1.5*S, by=0.5)
delta.call = BlackScholesValuation(S=S.shift , K=K, T=T, vola=vola, rf=rf)$call.delta
delta.put = BlackScholesValuation(S=S.shift , K=K, T=T, vola=vola, rf=rf)$put.delta

result.delta <- data.frame(S.shift = S.shift, delta.call = delta.call, delta.put = delta.put)

ggplot(result.delta, aes(x=S.shift)) + 
  geom_line(aes(y = delta.call), color = "darkred") + 
geom_line(aes(y = delta.put), color = "blue") 


# Implied Vola
option.price = 10
implied.vola <- findImpliedVola(option.price, S, K, rf, T, option.type="call")
print(paste("Implied Vola with Option price:",option.price,"CHF","is:",round(implied.vola,5)))

In [ ]:

# MONTE CARLO EUROPEAN - EULER
result.df.euler <- NULL ; result.sim <- NULL

for(i in 1:10^4) {
  result.sim <- MCEuropeanOptionValuation(S=S, K=K, dt=1, T=T, vola=vola, rf=rf, num.simulations=i, method="Euler")
  result.df.euler <- rbind(result.df.euler, result.sim)
}
result.df.euler$call.error <- result.df.euler$call.value - BS.option.value$call.value
result.df.euler$put.error <- result.df.euler$put.value - BS.option.value$put.value
result.df.euler[10000,]

plot(result.df.euler$call.error, type="l", col="red", main="Monte Carlo (Euler) vs Black Scholes - 10k sim", xlab="Number of Simulations", ylab="Valuation Error")
abline( h = 0, col = "gray40")

plot(result.df.euler$call.error[1:100], type="l", col="red", main="Monte Carlo (Euler) vs Black Scholes - 100 sim", xlab="Number of Simulations", ylab="Valuation Error")
abline( h = 0, col = "gray40")

#######################################

# MONTE CARLO EUROPEAN - ITO
result.df.ito <- NULL ; result.sim <- NULL
for(i in 1:10^4) {
  result.sim <- MCEuropeanOptionValuation(S=S, K=K, dt=1, T=T, vola=vola, rf=rf, num.simulations=i, method="Ito")
  result.df.ito <- rbind(result.df.ito, result.sim)
}
result.df.ito$call.error <- result.df.ito$call.value - BS.option.value$call.value
result.df.ito$put.error  <- result.df.ito$put.value - BS.option.value$put.value
result.df.ito[10000,]

plot(result.df.ito$call.error, type="l", col="red", main="Monte Carlo (Ito) vs Black Scholes - 10k sim", xlab="Number of Simulations", ylab="Valuation Error")
abline( h = 0, col = "gray40")


#######################################

single.simulation.df <- EulerPathSimulator(num.steps=252, num.simulations=1, num.discrete.steps=18, S=S, vola=vola, rf=rf)

plot(single.simulation.df$S.cont, type="l",
     main="Simulated Price Path",xlab="Days",ylab="Price", col="blue")
lines(x=single.simulation.df$S.discrete,col="red",type="l")
legend(x="topleft", legend=c("Continuous", "Discrete"),
       col=c("blue", "red"), lty=1:1, cex=0.8, box.lty=1)

plot(single.simulation.df$S.cont, type="l",
     main="Simulated Price Path",xlab="Days",ylab="Price", col="blue")
lines(x=single.simulation.df$S.cont.mean.arith,col="red",type="l")
lines(x=single.simulation.df$S.cont.mean.geom,col="green",type="l")
legend(x="topleft", legend=c("Geometric mean", "Arithmetic mean"),
       col=c("green", "red"), lty=1:1, cex=0.8, box.lty=1)

#######################################

# ASIAN OPTION VALUATION

num.simulations =10^4
num.steps = 252
sim.df <- EulerPathSimulator(num.steps=num.steps, num.simulations=num.simulations, num.discrete.steps=18, S=S, vola=vola, rf=rf)

summary.sim.df <- sim.df[sim.df$day==num.steps,]

# fixed strike

summary.sim.df$asian.call.fixed.arith.cont <- exp(-rf*T)*pmax(summary.sim.df$S.cont.mean.arith-K,0)
summary.sim.df$asian.call.fixed.geom.cont  <- exp(-rf*T)*pmax(summary.sim.df$S.cont.mean.geom-K,0)
summary.sim.df$asian.put.fixed.arith.cont  <- exp(-rf*T)*pmax(K-summary.sim.df$S.cont.mean.arith,0)
summary.sim.df$asian.put.fixed.geom.cont   <- exp(-rf*T)*pmax(K-summary.sim.df$S.cont.mean.geom,0)

summary.sim.df$asian.call.fixed.arith.disc <- exp(-rf*T)*pmax(summary.sim.df$S.discrete.mean.arith-K,0)
summary.sim.df$asian.call.fixed.geom.disc  <- exp(-rf*T)*pmax(summary.sim.df$S.discrete.mean.geom-K,0)
summary.sim.df$asian.put.fixed.arith.disc  <- exp(-rf*T)*pmax(K-summary.sim.df$S.discrete.mean.arith,0)
summary.sim.df$asian.put.fixed.geom.disc   <- exp(-rf*T)*pmax(K-summary.sim.df$S.discrete.mean.geom,0)



# floating strike

summary.sim.df$asian.call.floating.arith.cont <- exp(-rf*T)*pmax(summary.sim.df$S.cont-summary.sim.df$S.cont.mean.arith,0)
summary.sim.df$asian.call.floating.geom.cont  <- exp(-rf*T)*pmax(summary.sim.df$S.cont-summary.sim.df$S.cont.mean.geom,0)
summary.sim.df$asian.put.floating.arith.cont  <- exp(-rf*T)*pmax(summary.sim.df$S.cont.mean.arith-summary.sim.df$S.cont,0)
summary.sim.df$asian.put.floating.geom.cont   <- exp(-rf*T)*pmax(summary.sim.df$S.cont.mean.geom-summary.sim.df$S.cont,0)

summary.sim.df$asian.call.floating.arith.disc <- exp(-rf*T)*pmax(summary.sim.df$S.discrete-summary.sim.df$S.discrete.mean.arith,0)
summary.sim.df$asian.call.floating.geom.disc  <- exp(-rf*T)*pmax(summary.sim.df$S.discrete-summary.sim.df$S.discrete.mean.geom,0)
summary.sim.df$asian.put.floating.arith.disc  <- exp(-rf*T)*pmax(summary.sim.df$S.discrete.mean.arith-summary.sim.df$S.discrete,0)
summary.sim.df$asian.put.floating.geom.disc   <- exp(-rf*T)*pmax(summary.sim.df$S.discrete.mean.geom-summary.sim.df$S.discrete,0)


asian.option.fixed.valuation.summary <- data.frame(num.simulations = max(summary.sim.df$id.simulation),
                                     asian.call.fixed.arith.cont = mean(summary.sim.df$asian.call.fixed.arith.cont),
                                     asian.call.fixed.geom.cont  = mean(summary.sim.df$asian.call.fixed.geom.cont),
                                     asian.put.fixed.arith.cont  = mean(summary.sim.df$asian.put.fixed.arith.cont),
                                     asian.put.fixed.geom.cont   = mean(summary.sim.df$asian.put.fixed.geom.cont),
                                     asian.call.fixed.arith.disc = mean(summary.sim.df$asian.call.fixed.arith.disc),
                                     asian.call.fixed.geom.disc  = mean(summary.sim.df$asian.call.fixed.geom.disc),
                                     asian.put.fixed.arith.disc  = mean(summary.sim.df$asian.put.fixed.arith.disc),
                                     asian.put.fixed.geom.disc   = mean(summary.sim.df$asian.put.fixed.geom.disc)
                                      )



asian.option.floating.valuation.summary <- data.frame(num.simulations = max(summary.sim.df$id.simulation),
                                                      asian.call.floating.arith.cont = mean(summary.sim.df$asian.call.floating.arith.cont),
                                                      asian.call.floating.geom.cont  = mean(summary.sim.df$asian.call.floating.geom.cont),
                                                      asian.put.floating.arith.cont  = mean(summary.sim.df$asian.put.floating.arith.cont),
                                                      asian.put.floating.geom.cont   = mean(summary.sim.df$asian.put.floating.geom.cont),
                                                      asian.call.floating.arith.disc = mean(summary.sim.df$asian.call.floating.arith.disc),
                                                      asian.call.floating.geom.disc  = mean(summary.sim.df$asian.call.floating.geom.disc),
                                                      asian.put.floating.arith.disc  = mean(summary.sim.df$asian.put.floating.arith.disc),
                                                      asian.put.floating.geom.disc   = mean(summary.sim.df$asian.put.floating.geom.disc)
                                                       )

